
# Clase 5 — Comparación **K-NN** vs **Árbol de Decisión** (solo visto en clase)

**Entorno:** Anaconda + Jupyter  
**Dataset:** `Iris` de scikit-learn  
**Consigna:** Entrenar, evaluar y comparar la **precisión** de K-NN y Árbol de Decisión sobre un conjunto de datos.

> Esta versión respeta estrictamente lo visto en clase: `train_test_split`, `KNeighborsClassifier`, `DecisionTreeClassifier`, `accuracy_score`, `confusion_matrix`, `plot_tree` y gráficos con `matplotlib`.


In [ ]:

# 1) Imports basicos
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score, confusion_matrix

# Reproducibilidad
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)


In [ ]:

# 2) Carga de IRIS y seleccion de 2 caracteristicas para visualizar (largo/ancho del sepalo)
iris = load_iris()
X_full = pd.DataFrame(iris.data, columns=iris.feature_names)
y = pd.Series(iris.target, name="target")

# Usamos solo las dos primeras columnas tal como se muestra en la clase
X = X_full.iloc[:, :2].copy()  # sepal length (cm), sepal width (cm)

print("Clases:", dict(enumerate(iris.target_names)))
print("Features usadas:", X.columns.tolist())
X.head()


In [ ]:

# 3) Train/Test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=RANDOM_STATE, stratify=y
)

X_train.shape, X_test.shape


In [ ]:

# 4) K-NN con k=15 (uniform vs distance)
knn_uniform = KNeighborsClassifier(n_neighbors=15, weights='uniform')
knn_distance = KNeighborsClassifier(n_neighbors=15, weights='distance')

knn_uniform.fit(X_train, y_train)
knn_distance.fit(X_train, y_train)

y_pred_knn_u = knn_uniform.predict(X_test)
y_pred_knn_d = knn_distance.predict(X_test)

acc_knn_u = accuracy_score(y_test, y_pred_knn_u)
acc_knn_d = accuracy_score(y_test, y_pred_knn_d)

cm_knn_u = confusion_matrix(y_test, y_pred_knn_u)
cm_knn_d = confusion_matrix(y_test, y_pred_knn_d)

print(f"Accuracy Test KNN (uniform):  {acc_knn_u:.4f}")
print(f"Accuracy Test KNN (distance): {acc_knn_d:.4f}\n")

print("Matriz de confusion KNN (uniform):\n", cm_knn_u)
print("Matriz de confusion KNN (distance):\n", cm_knn_d)


In [ ]:

# 5) Regiones de decision para KNN (uniform y distance) con meshgrid
h = 0.05  # paso de la grilla
x_min, x_max = X.iloc[:,0].min() - 1, X.iloc[:,0].max() + 1
y_min, y_max = X.iloc[:,1].min() - 1, X.iloc[:,1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))

# uniform
Z_u = knn_uniform.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
plt.figure()
plt.contourf(xx, yy, Z_u, alpha=0.3)
plt.scatter(X.iloc[:,0], X.iloc[:,1], c=y, edgecolor="k")
plt.title("KNN (k=15, weights='uniform') — Regiones de decision")
plt.xlabel(X.columns[0]); plt.ylabel(X.columns[1])
plt.show()

# distance
Z_d = knn_distance.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
plt.figure()
plt.contourf(xx, yy, Z_d, alpha=0.3)
plt.scatter(X.iloc[:,0], X.iloc[:,1], c=y, edgecolor="k")
plt.title("KNN (k=15, weights='distance') — Regiones de decision")
plt.xlabel(X.columns[0]); plt.ylabel(X.columns[1])
plt.show()


In [ ]:

# 6) Arbol de Decision (max_depth limitado para evitar sobreajuste)
tree = DecisionTreeClassifier(max_depth=4, random_state=RANDOM_STATE)
tree.fit(X_train, y_train)
y_pred_tree = tree.predict(X_test)

acc_tree = accuracy_score(y_test, y_pred_tree)
cm_tree = confusion_matrix(y_test, y_pred_tree)

print(f"Accuracy Test Arbol: {acc_tree:.4f}")
print("Matriz de confusion Arbol:\n", cm_tree)


In [ ]:

# 7) Visualizacion del arbol y de las importancias
plt.figure(figsize=(10, 8), dpi=150)
plot_tree(tree, feature_names=X.columns, class_names=iris.target_names, filled=True)
plt.title("Arbol de Decision (max_depth=4)")
plt.show()

importances = tree.feature_importances_
plt.figure()
plt.bar(X.columns, importances)
plt.ylabel("Importancia")
plt.title("Importancia de caracteristicas (Arbol)")
plt.show()


In [ ]:

# 8) Comparacion simple de accuracy
modelos = ["KNN (uniform)", "KNN (distance)", "Arbol"]
accuracies = [acc_knn_u, acc_knn_d, acc_tree]

plt.figure()
plt.bar(modelos, accuracies)
plt.ylim(0,1)
plt.title("Accuracy en Test - Comparacion")
plt.show()

pd.DataFrame({"Modelo": modelos, "Accuracy_Test": accuracies})



## Conclusiones (para el README)
- **Rendimiento:** compara los valores de *accuracy* de test y comentá cuál resultó mejor en este dataset y por qué podría ocurrir.
- **K-NN:** `k=15` y `weights` afectan el resultado; es sensible a la escala (aquí no escalamos por mantenernos en lo visto).
- **Árbol:** con `max_depth=4` evitamos árboles demasiado profundos; permite interpretar reglas y ver importancias.
- **Decisión informada:** elegí el modelo según el equilibrio entre rendimiento, interpretabilidad y requisitos de preprocesamiento.
